In [3]:
import pandas as pd
import altair as alt

# Load your data
df = pd.read_csv('15000_tracks_cleaned.csv')

# Filter to your specific genres
main_genres = ['acoustic', 'afrobeat', 'ambient', 'bluegrass', 'chicago-house',
               'children', 'club', 'comedy']
df_filtered = df[df['track_genre'].isin(main_genres)].copy()

# Create color scheme matching your specifications
genre_colors = alt.Scale(
    domain=['acoustic', 'afrobeat', 'ambient', 'bluegrass', 'chicago-house',
            'children', 'club', 'comedy'],
    range=['#FFCA8E', '#40EDC8', '#F77B87', '#3CED19', '#662D91',
           '#FF9DF4', '#FDFF94', '#9BDEFF']
)

# Add decade slider for animation - min at 2000, default at 2020
decade_slider = alt.binding_range(min=2000,
                                   max=df_filtered['decade'].max(),
                                   step=10,
                                   name='Decade: ')
decade_select = alt.selection_point(name='decade_select',
                                     fields=['decade'],
                                     bind=decade_slider,
                                     value=2020)

# Add click selection for genres
genre_click = alt.selection_point(fields=['track_genre'],
                                   bind='legend',
                                   name='genre_selection')

# Create the animated scatter plot
chart = alt.Chart(df_filtered).mark_circle().encode(
    x=alt.X('energy:Q',
            scale=alt.Scale(domain=[0, 1]),
            title='Energy',
            axis=alt.Axis(grid=True, labelFontSize=13, titleFontSize=18)),
    y=alt.Y('danceability:Q',
            scale=alt.Scale(domain=[0, 1]),
            title='Danceability',
            axis=alt.Axis(grid=True, labelFontSize=13, titleFontSize=18)),
    size=alt.Size('popularity:Q',
                  scale=alt.Scale(range=[50, 1000]),
                  legend=alt.Legend(title='Popularity')),
    color=alt.Color('track_genre:N',
                    scale=genre_colors,
                    legend=alt.Legend(title='Genre')),
    tooltip=[
        alt.Tooltip('artists:N', title='Artist'),
        alt.Tooltip('track_name:N', title='Track'),
        alt.Tooltip('track_genre:N', title='Genre'),
        alt.Tooltip('year:Q', title='Year'),
        alt.Tooltip('energy:Q', format='.3f'),
        alt.Tooltip('danceability:Q', format='.3f'),
        alt.Tooltip('popularity:Q')
    ],
    opacity=alt.condition(genre_click, alt.value(0.7), alt.value(0.1))
).properties(
    width=800,
    height=600,
    title={
        "text": "Music Evolution: Genre Patterns Across Decades",
        "subtitle": "Bubble size represents popularity | Use slider to explore different time periods | Click legend to filter genres",
        "fontSize": 22,
        "subtitleFontSize": 13
    }
).add_params(
    alt.selection_interval(bind='scales', encodings=['x', 'y']),
    decade_select,
    genre_click
).transform_filter(
    decade_select
)

# Save as HTML
chart.save('music_evolution_animated.html')